In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import os
import codecs
import html2text
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from textblob import TextBlob
from nltk.corpus import stopwords
from wordcloud import WordCloud
import seaborn
from nltk import word_tokenize, pos_tag
import ast
from operator import itemgetter
from gensim.models import LdaModel
from scipy.sparse.linalg import svds
from gensim.models.phrases import Phrases, Phraser
from gensim.corpora import Dictionary
from gensim.models.nmf import Nmf
from gensim.models.coherencemodel import CoherenceModel
from sklearn.decomposition import NMF
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import AdaBoostRegressor
import numpy as np


In [ ]:
dataset1 = pd.read_excel("jester-data-1.xls")
dataset2 = pd.read_excel("jester-data-2.xls")
dataset3 = pd.read_excel("jester-data-3.xls")

def insert_return(frame):
    ret_lst = []
    for index,row in frame.iterrows():
        ret_lst.append(list(row))
    
    return ret_lst

def combine_dataframe(frame1, frame2, frame3):
    joke_lst = ["Number of jokes rated"]
    for i in range(100):
        joke_lst.append(f"joke-{i}")

    rating_lst = []

    rating_lst.extend(insert_return(frame1))
    rating_lst.extend(insert_return(frame2))
    rating_lst.extend(insert_return(frame3))

    return pd.DataFrame(data=rating_lst,columns=joke_lst)

In [ ]:
ratings = combine_dataframe(dataset1, dataset2,dataset3)

In [ ]:
ratings.replace(99.0, np.nan, inplace=True)
ratings.head()


,Number of jokes rated,joke-0,joke-1,joke-2,joke-3,joke-4,joke-5,joke-6,joke-7,joke-8,...,joke-90,joke-91,joke-92,joke-93,joke-94,joke-95,joke-96,joke-97,joke-98,joke-99
0,100.0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,49.0,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
2,48.0,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
3,91.0,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,100.0,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,...,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45


In [ ]:
jokes_dataframe = pd.DataFrame()

In [ ]:
def load_clean_joke():

    ret_jokes = []

    for i in range(1,101):
        file = 'init'+str(i)+'.html'
        data = codecs.open('jokes/' + file, 'r', encoding="cp1252")
        joke_html = data.read()

        # Extracting joke
        joke = html2text.html2text(joke_html)
        # Extracting joke_id
        joke_id = int(file.split('init')[1].split('.html')[0])
        cleaned_string = re.sub(r'[\|]+|[-]+', '', joke)

        ret_jokes.append(cleaned_string.strip())
    
    return ret_jokes


In [ ]:
Jokes = load_clean_joke()

In [ ]:
from nltk.corpus import stopwords

def remove_tags_puntuatuions_tags(Joke):
    special_char_patterns = r'[^a-zA-Z0-9\s]'
    urlPatterns = r'http\S+|www\S+'

    text = Joke

    text = re.sub(special_char_patterns, ' ', text)
    text = re.sub(urlPatterns, ' ', text)
    text = re.sub(r'\n', ' ', text)

    return text

def remove_stop_words_Tokenization(Joke):
    
    stopwords_ = set(stopwords.words('english'))
    Tokenize_Joke = []
    for word in Joke.split():
        if word not in stopwords_:
            Tokenize_Joke.append(word)

    return Tokenize_Joke

def lower(Joke):
    ret_lst = [word.lower() for word in Joke]
    return ret_lst

def preprocess_clean_jokes(uncleaned_Jokes):

    Cleaned_Jokes = []

    for Joke in uncleaned_Jokes:
        Joke = remove_tags_puntuatuions_tags(Joke)
        Joke = TextBlob(Joke)
        Joke = remove_stop_words_Tokenization(Joke)

        Cleaned_Jokes.append(lower(Joke))

    return Cleaned_Jokes



In [ ]:
preprocessed_jokes = preprocess_clean_jokes(Jokes)

In [ ]:
jokes_dataframe["PreProcessed Jokes"] = preprocessed_jokes

In [ ]:
from nltk.stem import PorterStemmer

def stem_tokens(tokenized_text):
    stemmer = PorterStemmer()
    stemmed_tokens = []
    for preprocessed_joke in tokenized_text:
        stemmed_tokens.append([stemmer.stem(token) for token in preprocessed_joke])
    
    return stemmed_tokens

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
        
    lemmatized_text = " ".join([lemmatizer.lemmatize(word) for word in text])
    
    return lemmatized_text

In [ ]:
lemmitized_jokes = [lemmatize_text(text) for text in preprocessed_jokes]

In [ ]:
jokes_dataframe["lemmitized_jokes"] = lemmitized_jokes 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3))
tfidf_matrix = tfidf_vectorizer.fit_transform(lemmitized_jokes)
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_array = tfidf_matrix.toarray()



In [ ]:
TF_IDF_lemma = tfidf_array

In [ ]:
jokes_dataframe["TF_IDF_Lemma"] = [i for i in TF_IDF_lemma] 

In [ ]:
cleaned_list = list(jokes_dataframe["PreProcessed Jokes"])

merged_lst = []

for joke in cleaned_list:
    merged_lst.append(" ".join(joke))



In [ ]:
jokes_dataframe["merged cleaned"] = merged_lst

In [ ]:
import nltk

def posTagging(text):
    return nltk.pos_tag(text, tagset='universal')

jokes_dataframe['pos_tags'] = jokes_dataframe['PreProcessed Jokes'].apply(lambda x: posTagging(x))
jokes_dataframe.head(2)

,PreProcessed Jokes,lemmitized_jokes,TF_IDF_Lemma,merged cleaned,pos_tags
0,"[a, man, visits, doctor, the, doctor, says, i,...",a man visit doctor the doctor say i bad news y...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a man visits doctor the doctor says i bad news...,"[(a, DET), (man, NOUN), (visits, VERB), (docto..."
1,"[this, couple, excellent, relationship, going,...",this couple excellent relationship going one d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",this couple excellent relationship going one d...,"[(this, DET), (couple, ADJ), (excellent, NOUN)..."


In [ ]:
jokes_dataframe

,PreProcessed Jokes,lemmitized_jokes,TF_IDF_Lemma,merged cleaned,pos_tags
0,"[a, man, visits, doctor, the, doctor, says, i,...",a man visit doctor the doctor say i bad news y...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a man visits doctor the doctor says i bad news...,"[(a, DET), (man, NOUN), (visits, VERB), (docto..."
1,"[this, couple, excellent, relationship, going,...",this couple excellent relationship going one d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",this couple excellent relationship going one d...,"[(this, DET), (couple, ADJ), (excellent, NOUN)..."
2,"[q, what, 200, feet, long, 4, teeth, a, the, f...",q what 200 foot long 4 teeth a the front row w...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what 200 feet long 4 teeth a the front row w...,"[(q, VERB), (what, PRON), (200, NUM), (feet, N..."
3,"[q, what, difference, man, toilet, a, a, toile...",q what difference man toilet a a toilet follow...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what difference man toilet a a toilet follow...,"[(q, VERB), (what, PRON), (difference, NOUN), ..."
4,"[q, what, o, j, simpson, internet, address, a,...",q what o j simpson internet address a slash sl...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what o j simpson internet address a slash sl...,"[(q, VERB), (what, PRON), (o, ADP), (j, NOUN),..."
...,...,...,...,...,...
95,"[two, attorneys, went, diner, ordered, two, dr...",two attorney went diner ordered two drink then...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",two attorneys went diner ordered two drinks th...,"[(two, NUM), (attorneys, NOUN), (went, VERB), ..."
96,"[a, teacher, explaining, class, different, lan...",a teacher explaining class different language ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a teacher explaining class different languages...,"[(a, DET), (teacher, NOUN), (explaining, VERB)..."
97,"[age, womanhood, 1, between, ages, 13, 18, she...",age womanhood 1 between age 13 18 she like afr...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",age womanhood 1 between ages 13 18 she like af...,"[(age, NOUN), (womanhood, VERB), (1, NUM), (be..."
98,"[a, bus, station, bus, stops, a, train, statio...",a bus station bus stop a train station train s...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a bus station bus stops a train station train ...,"[(a, DET), (bus, NOUN), (station, NOUN), (bus,..."


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
import spacy
import textstat

def detect_irony(joke):
    blob = TextBlob(joke)
    
    irony_detected = blob.sentiment.polarity < 0
    
    return irony_detected

def extract_humor_features(joke):
    nlp = spacy.load("en_core_web_sm")

    doc = nlp(joke)

    wordplay_detected = False
    for token in doc:
        if token.lemma_.lower() == "play" and token.pos_ == "NOUN":
            wordplay_detected = True
            break

    incongruity_detected = False
    for token in doc:
        if token.dep_ == "prep" and token.head.pos_ == "NOUN":
            incongruity_detected = True
            break

    return wordplay_detected, incongruity_detected

def analyze_sentiment(joke):
    sia = SentimentIntensityAnalyzer()
    scores = sia.polarity_scores(joke)
    
    compound_score = scores['compound']
    sentiment = 'positive' if compound_score > 0 else ('negative' if compound_score < 0 else 'neutral')
    
    return scores['neg'], scores['pos'], scores['neu']

def extract_structure_features(joke):
    sentences = sent_tokenize(joke)

    sentence_lengths = [len(word_tokenize(sentence)) for sentence in sentences]
    avg_sentence_length = sum(sentence_lengths) / len(sentences)
    max_sentence_length = max(sentence_lengths)

    tokenizer = RegexpTokenizer(r'\w+|\S')
    tokens = tokenizer.tokenize(joke)
    punctuation_count = len([token for token in tokens if token.isalnum() == False])

    paragraphs = joke.split('\n')
    paragraph_count = len(paragraphs)

    return avg_sentence_length, max_sentence_length, punctuation_count, paragraph_count

def extract_pos_and_grammar_features(joke):
    nlp = spacy.load("en_core_web_sm")

    doc = nlp(joke)

    pos_distribution = {token.pos_: pos_count for pos_count, token in enumerate(doc)}

    grammar_quality = "Good" if not any(token.is_space for token in doc if token.pos_ == "VERB") else "Poor"

    pos_lst = ['SPACE', 'SCONJ', 'AUX', 'DET', 'NOUN', 'VERB', 'PUNCT', 'PRON', 'ADJ', 'ADP']

    for pos in pos_lst:
        if pos not in pos_distribution.keys():
            pos_distribution[pos] = 0

    return pos_distribution, grammar_quality

def extract_lexical_features(joke):
    nlp = spacy.load("en_core_web_sm")

    doc = nlp(joke)

    vocabulary = [token.text.lower() for token in doc if token.is_alpha]
    vocabulary_richness = len(set(vocabulary)) / len(vocabulary) if len(vocabulary) > 0 else 0

    lexical_diversity = len(set(vocabulary)) / len(doc) if len(doc) > 0 else 0

    return vocabulary_richness, lexical_diversity

def extract_stylistic_features(joke):
    nlp = spacy.load("en_core_web_sm")

    doc = nlp(joke)

    readability_scores = {
        'flesch_reading_ease': textstat.flesch_reading_ease(joke),
        'flesch_kincaid_grade': textstat.flesch_kincaid_grade(joke),
        'automated_readability_index': textstat.automated_readability_index(joke),
    }

    jargon_slang_detected = any(token.is_alpha and token.text.lower() in ["jargon", "slang"] for token in doc)

    return readability_scores, jargon_slang_detected

def feature_Extraction(Org_Jokes, Lemma_Jokes):
    
    Jokes_Features = []

    column_names = ['neg_content', 'pos_content', 'neu_content', 'avg_sentence_length', 'max_sentence_length',
                'punctuation_count', 'paragraph_count', 'wordplay_detected', 'incongruity_detected',
                'len_filtered_words', 'len_set_filtered_words', 'pos_distribution_VERB', 'pos_distribution_NOUN',
                'pos_distribution_ADJ', 'vocabulary_richness', 'lexical_diversity', 'jargon_slang_detected',
                'flesch_reading_ease', 'flesch_kincaid_grade', 'automated_readability_index'
                , 'irony']

    for Org_Joke, Lemma_Joke in zip(Org_Jokes, Lemma_Jokes):
        
        Org_Joke = ' '.join(Org_Joke)
        words = word_tokenize(Org_Joke)

        stop_words = set(stopwords.words('english'))
        filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]

        neg_content, pos_content, neu_content = analyze_sentiment(Org_Joke)
        avg_sentence_length, max_sentence_length, punctuation_count, paragraph_count = extract_structure_features(Org_Joke)
        wordplay_detected, incongruity_detected = extract_humor_features(Org_Joke)
        pos_distribution, grammar_quality = extract_pos_and_grammar_features(Lemma_Joke)
        vocabulary_richness, lexical_diversity = extract_lexical_features(Lemma_Joke)
        readability_scores, jargon_slang_detected = extract_stylistic_features(Org_Joke)
        irony =detect_irony(Lemma_Joke)

        Jokes_Features.append([neg_content, pos_content, neu_content, avg_sentence_length, max_sentence_length, punctuation_count, paragraph_count
    , wordplay_detected, incongruity_detected, len(filtered_words), len(set(filtered_words)), 
    pos_distribution["VERB"], pos_distribution["NOUN"],  pos_distribution["ADJ"], vocabulary_richness, lexical_diversity,
    jargon_slang_detected, readability_scores['flesch_reading_ease'], readability_scores['flesch_kincaid_grade'],
    readability_scores['automated_readability_index'],irony
    ])

    return pd.DataFrame(Jokes_Features, columns=column_names)



In [ ]:
features_Jokes = feature_Extraction(list(jokes_dataframe["PreProcessed Jokes"]), list(jokes_dataframe["lemmitized_jokes"]))

In [ ]:
features_Jokes

,neg_content,pos_content,neu_content,avg_sentence_length,max_sentence_length,punctuation_count,paragraph_count,wordplay_detected,incongruity_detected,len_filtered_words,...,pos_distribution_VERB,pos_distribution_NOUN,pos_distribution_ADJ,vocabulary_richness,lexical_diversity,jargon_slang_detected,flesch_reading_ease,flesch_kincaid_grade,automated_readability_index,irony
0,0.384,0.209,0.406,22.0,22,0,1,False,False,16,...,18,21,8,0.772727,0.772727,False,74.53,8.3,9.7,True
1,0.070,0.087,0.843,38.0,38,0,1,False,False,32,...,30,36,37,0.947368,0.947368,False,49.83,15.8,21.4,False
2,0.000,0.000,1.000,14.0,14,0,1,False,False,11,...,13,11,0,1.000000,0.857143,False,99.57,2.9,3.4,True
3,0.000,0.000,1.000,11.0,11,0,1,False,False,8,...,4,10,0,0.818182,0.818182,False,68.77,6.4,4.2,True
4,0.556,0.113,0.331,14.0,14,0,1,False,False,11,...,0,13,0,0.785714,0.785714,False,74.19,6.4,7.4,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000,0.000,1.000,30.0,30,0,1,False,False,26,...,28,29,26,0.800000,0.800000,False,32.57,16.2,23.1,False
96,0.278,0.238,0.483,44.0,44,0,1,False,False,39,...,41,40,36,0.568182,0.568182,False,1.44,24.0,29.6,True
97,0.074,0.327,0.599,57.0,57,0,1,False,True,48,...,55,47,34,0.674419,0.508772,False,22.08,24.3,29.6,False
98,0.242,0.000,0.758,15.0,15,0,1,False,True,11,...,13,14,0,0.600000,0.600000,False,90.09,4.4,4.9,False


In [ ]:
features_Jokes = features_Jokes.astype(int)

In [ ]:
features_Jokes

,neg_content,pos_content,neu_content,avg_sentence_length,max_sentence_length,punctuation_count,paragraph_count,wordplay_detected,incongruity_detected,len_filtered_words,...,pos_distribution_VERB,pos_distribution_NOUN,pos_distribution_ADJ,vocabulary_richness,lexical_diversity,jargon_slang_detected,flesch_reading_ease,flesch_kincaid_grade,automated_readability_index,irony
0,0,0,0,22,22,0,1,0,0,16,...,18,21,8,0,0,0,74,8,9,1
1,0,0,0,38,38,0,1,0,0,32,...,30,36,37,0,0,0,49,15,21,0
2,0,0,1,14,14,0,1,0,0,11,...,13,11,0,1,0,0,99,2,3,1
3,0,0,1,11,11,0,1,0,0,8,...,4,10,0,0,0,0,68,6,4,1
4,0,0,0,14,14,0,1,0,0,11,...,0,13,0,0,0,0,74,6,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,1,30,30,0,1,0,0,26,...,28,29,26,0,0,0,32,16,23,0
96,0,0,0,44,44,0,1,0,0,39,...,41,40,36,0,0,0,1,24,29,1
97,0,0,0,57,57,0,1,0,1,48,...,55,47,34,0,0,0,22,24,29,0
98,0,0,0,15,15,0,1,0,1,11,...,13,14,0,0,0,0,90,4,4,0


In [ ]:
dataset1 = pd.read_excel("jester-data-1.xls")
dataset2 = pd.read_excel("jester-data-2.xls")
dataset3 = pd.read_excel("jester-data-3.xls")

def insert_return(frame):
    ret_lst = []
    for index,row in frame.iterrows():
        ret_lst.append(list(row))
    
    return ret_lst

def combine_dataframe(frame1, frame2, frame3):
    joke_lst = ["Number of jokes rated"]
    for i in range(100):
        joke_lst.append(f"joke-{i}")

    rating_lst = []

    rating_lst.extend(insert_return(frame1))
    rating_lst.extend(insert_return(frame2))
    rating_lst.extend(insert_return(frame3))

    return pd.DataFrame(data=rating_lst,columns=joke_lst)

In [ ]:
ratings = combine_dataframe(dataset1, dataset2,dataset3)
ratings.replace(99.0, np.nan, inplace=True)
ratings.head()

,Number of jokes rated,joke-0,joke-1,joke-2,joke-3,joke-4,joke-5,joke-6,joke-7,joke-8,...,joke-90,joke-91,joke-92,joke-93,joke-94,joke-95,joke-96,joke-97,joke-98,joke-99
0,100.0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,49.0,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
2,48.0,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
3,91.0,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,100.0,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,...,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45


In [ ]:
column_averages = ratings.mean()

In [ ]:
ratings_jokes = list(column_averages)[1:]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features_Jokes.to_numpy(), ratings_jokes, test_size=0.10)

In [ ]:
X_train = features_Jokes.to_numpy()
Y_train = ratings_jokes

In [ ]:
regressor = AdaBoostRegressor(random_state=42,n_estimators=20)

kf = KFold(n_splits=3, shuffle=True, random_state=42)

r2_scores = cross_val_score(regressor, X_train, Y_train, cv=kf, scoring='r2')

for i, score in enumerate(r2_scores, 1):
    print(f'Fold {i}: R2 Score = {score}')

average_r2 = np.mean(r2_scores)
print(f'Average R2 Score: {average_r2}')


Fold 1: R2 Score = 0.17253003213865992
Fold 2: R2 Score = 0.25543560999061543
Fold 3: R2 Score = 0.4707202334547642
Average R2 Score: 0.2995619585280132


In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score

regressor = SVR()

kf = KFold(n_splits=3, shuffle=True, random_state=42)

r2_scores = cross_val_score(regressor, X_train, Y_train, cv=kf, scoring='r2')

for i, score in enumerate(r2_scores, 1):
    print(f'Fold {i}: R2 Score = {score}')

average_r2 = np.mean(r2_scores)
print(f'Average R2 Score: {average_r2}')


Fold 1: R2 Score = 0.3523324121256475
Fold 2: R2 Score = 0.24880533943538685
Fold 3: R2 Score = 0.43750078616873
Average R2 Score: 0.34621284590992146


In [ ]:
from sklearn.linear_model import Ridge

alpha = 10
ridge_regressor = Ridge(alpha=alpha)

kf = KFold(n_splits=3, shuffle=True, random_state=42)

r2_scores_ridge = cross_val_score(ridge_regressor, X_train, Y_train, cv=kf, scoring='r2')

for i, score in enumerate(r2_scores_ridge, 1):
    print(f'Ridge Regression - Fold {i}: R2 Score = {score}')

average_r2_ridge = np.mean(r2_scores_ridge)
print(f'Average R2 Score (Ridge Regression): {average_r2_ridge}')


Ridge Regression - Fold 1: R2 Score = 0.28570630734033986
Ridge Regression - Fold 2: R2 Score = 0.25435674361082017
Ridge Regression - Fold 3: R2 Score = 0.28012271624086105
Average R2 Score (Ridge Regression): 0.2733952557306737


In [ ]:
from sklearn.linear_model import BayesianRidge

bayesian_regressor = BayesianRidge()

kf = KFold(n_splits=3, shuffle=True, random_state=42)

r2_scores_bayesian = cross_val_score(bayesian_regressor, X_train, Y_train, cv=kf, scoring='r2')

for i, score in enumerate(r2_scores_bayesian, 1):
    print(f'Bayesian Ridge Regression - Fold {i}: R2 Score = {score}')

average_r2_bayesian = np.mean(r2_scores_bayesian)
print(f'Average R2 Score (Bayesian Ridge Regression): {average_r2_bayesian}')


Bayesian Ridge Regression - Fold 1: R2 Score = 0.3409172080711239
Bayesian Ridge Regression - Fold 2: R2 Score = 0.2824460394493197
Bayesian Ridge Regression - Fold 3: R2 Score = 0.43033910865836966
Average R2 Score (Bayesian Ridge Regression): 0.3512341187262711


In [ ]:
from sklearn.svm import SVR

Y_train = np.array(Y_train)

C_value = 3
svr_regressor = SVR(C=C_value)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
models = []

k_fold_score = []
predictions = []

for i, (train_index, val_index) in enumerate(kf.split(X_train, Y_train), 1):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]

    svr_regressor.fit(X_train_fold, Y_train_fold)
    models.append(svr_regressor)
    predictions_val = svr_regressor.predict(X_val_fold)

    r2_score_fold = r2_score(y_true=Y_val_fold, y_pred=predictions_val)
    k_fold_score.append(r2_score_fold)
    


    print(f'SVR - Fold {i}: R2 Score = {r2_score_fold}')

print("\n")
print(f"Testing result: {np.mean(k_fold_score)}")

SVR - Fold 1: R2 Score = 0.3597122787161956
SVR - Fold 2: R2 Score = 0.29118920953727145
SVR - Fold 3: R2 Score = 0.40012432453349733


Testing result: 0.35034193759565485


In [ ]:
# from sklearn.metrics import r2_score

# svr_Model = SVR(C=1)
# svr_Model.fit(X_train, Y_train)
# prediction = svr_Model.predict(X_test)
# score = r2_score(Y_test,prediction)
# print(f"Final Score {score}")

In [ ]:
# def calculate_r2(y_true, y_pred):
#     mean_y_true = np.mean(y_true)
    
#     sst = np.sum((y_true - mean_y_true)**2)
    
#     ssr = np.sum((y_true - y_pred)**2)
    
#     r2 = 1 - (ssr / sst)
    
#     return r2

# r2_score = calculate_r2(y_true=np.array(Y_test), y_pred=prediction)
# print(f'R2 Score: {r2_score}')


In [ ]:
def read_jokes_from_file(file_path):
    jokes = []

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                jokes.append(line.strip())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return jokes

file_path = '/content/test_joke.txt'
jokes_list = read_jokes_from_file(file_path)

File not found: /content/test_joke.txt


In [ ]:
def testing_jokes(jokes, model):
    preprocessed_test = preprocess_clean_jokes(jokes)
    lemmitized_test = [lemmatize_text(joke) for joke in preprocessed_jokes]
    extracted_test = feature_Extraction(preprocessed_test, lemmitized_test)
    extracted_test = extracted_test.astype(int)
    extracted_test.columns = [None] * len(extracted_test.columns)
    extracted_test = extracted_test.to_numpy()
    return model.predict(extracted_test)

In [ ]:
svr_Model = SVR(C=2)
svr_Model.fit(X_train, Y_train)
prediction = svr_Model.predict(X_test)
predictions = testing_jokes(jokes_list,svr_Model)

In [ ]:
predictions